In [ ]:
# %%capture -> decommend if you dont want to see outputs
%pip install rdkit
# ==========================================================================================
# =================================== IMPORTS AND SETUP ====================================
# ==========================================================================================

import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import optuna
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold
from sklearn.metrics import mean_absolute_error, make_scorer, mean_squared_error

from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen, rdMolDescriptors, MACCSkeys, RDKFingerprint, rdFingerprintGenerator
from rdkit.Chem.AtomPairs import Pairs, Torsions

# DISABLE WARNING FROM rdkit
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

try:
    from rdkit.Avalon import pyAvalonTools
    avalon_available = True
except ImportError:
    avalon_available = False

import plotly.io as pio
pio.renderers.default = "iframe_connected"

from IPython.display import clear_output

import xgboost

# ==========================================================================================
# ==================================== LOADING DATASETS ====================================
# ==========================================================================================

train_df = pd.read_csv("train.csv")
test_df  = pd.read_csv("test.csv")
bradley_df = pd.read_excel("helper/BradleyMeltingPointDataset.xlsx")
bradleyplus_df = pd.read_excel("helper/BradleyDoublePlusGoodMeltingPointDataset.xlsx")


train_df = train_df[['SMILES', 'Tm']]
test_df  = test_df[['id', 'SMILES']]

train_df.shape, test_df.shape, bradley_df.shape, bradleyplus_df.shape

display(train_df)
display(test_df)

# ==========================================================================================
# ========================== PREPROCESSING EXTERNAL DATA (BRADLEY) =========================
# ==========================================================================================

# CONVERT MELTING POINT CELCIUS TO KELVIN (Tm)
bradley_df['Tm'] = bradley_df['mpC'].map(lambda x : x + 273.15)
bradleyplus_df['Tm'] = bradleyplus_df['mpC'].map(lambda x: x + 273.15)

# GET ONLY SMILES AND Tm COLUMNS
bradley_df = bradley_df[['smiles', 'Tm']]
bradleyplus_df = bradleyplus_df[['smiles', 'Tm']]

# MERGE THEM
bradley_merge = pd.concat((bradley_df, bradleyplus_df), axis = 0)
bradley_merge = bradley_merge.rename(columns = {'smiles' : 'SMILES'})

bradley_merge

# ==========================================================================================
# =============================== MERGING AND CLEANING DATA ================================
# ==========================================================================================

# CONCAT TRAIN DATA AND EXTRA DATA
merge_df = pd.concat((train_df, bradley_merge), axis = 0)

merge_df

# DROP DUPLICATED FROM MERGED DATA
display(f'There are {merge_df.duplicated(subset = ["SMILES", "Tm"]).sum()} Duplicated data')

merge_df = merge_df.drop_duplicates(subset = ['SMILES', 'Tm']).reset_index(drop = True)

print(f'Successfully Drop Duplicated data!')

merge_df

# ==========================================================================================
# =========================== FEATURE ENGINEERING: DESCRIPTORS =============================
# ==========================================================================================

# EXTRACT ALL DESCRIPTORS FEATURES
def extract_all_descriptors(df, SMILES):

    # GET ALL DESCRIPTORS
    descriptor_list = Descriptors._descList    # --> THESE WILL RETURN LIST OF TUPLE
    descriptors = [desc[0] for desc in descriptor_list]

    print(f'There Are {len(descriptor_list)} Descriptor Features')

    # EXTRACT ALL DESCRIPTORS FROM SMILES FEATURES
    result = []
    for smi in df[SMILES]:

        mol = Chem.MolFromSmiles(smi)

        # IF MOLECOLE IS INVALID
        if mol is None:
            row = {name : None for name, func in descriptor_list}
        else:
            # CREATE DESCRIPTORS FEATURES
            row = {name: func(mol) for name, func in descriptor_list}

        result.append(row)

    # MERGE DATA WITH EXTRACTED FEATURES
    df_descriptor = pd.DataFrame(result)
    df_result = pd.concat((df, df_descriptor), axis = 1)

    return df_result


merge_df = extract_all_descriptors(merge_df, 'SMILES')
test_df  = extract_all_descriptors(test_df, 'SMILES')

# DROP NULL/NONE VALUE AFTER FEATURE ENGINEERING
merge_df = merge_df.dropna().reset_index(drop = True)
test_df = test_df.dropna().reset_index(drop = True)

merge_df

# ==========================================================================================
# =========================== FEATURE ENGINEERING: FINGERPRINTS ============================
# ==========================================================================================

# EXTRACT ALL MOLECULAR FINGERPRINT FEATURES
def extract_all_fingerprint(df, SMILES, morgan_radius = 2, morgan_nbits = 1024):

    fps_data = []  # --> STORE NEW FEATURES DATA

    # DEFINE MORGAN GENERATOR
    morgan_gen = rdFingerprintGenerator.GetMorganGenerator(radius = morgan_radius, fpSize = morgan_nbits, countSimulation = True, includeChirality = False)

    fcfp = rdFingerprintGenerator.GetMorganFeatureAtomInvGen()
    fcfp_gen = rdFingerprintGenerator.GetMorganGenerator(radius = morgan_nbits, fpSize = morgan_nbits, atomInvariantsGenerator = fcfp, countSimulation= True, includeChirality = False)

    atom_gen = rdFingerprintGenerator.GetAtomPairGenerator(fpSize = 2048, countSimulation= True, includeChirality = False)

    # ITERATE EVERY SAMPLE OF SMILES FEATURES
    for smiles in df[SMILES]:
        mol = Chem.MolFromSmiles(smiles)

        if mol is None:
            print(smiles, 'is Invalid!')
            fps_data.append({})
            continue

        # STORE NEW FEATURE FOR EACH SAMPLES CREATED
        feature_rows = {}

        # MORGAN FINGERPRINT (ECFP)
        morgan_fp = morgan_gen.GetFingerprint(mol)
        for i in range(morgan_nbits):
            feature_rows[f"Morgan_{i}"] = morgan_fp[i]

        # FUNCTIONAL-CLASS FINGERPRINT (FCFP)
        fc_fp = fcfp_gen.GetFingerprint(mol)
        for i in range(morgan_nbits):
            feature_rows[f"FCFP_{i}"] = fc_fp[i]

        # MACCS KEYS (166 BITS)
        maccs_fp = MACCSkeys.GenMACCSKeys(mol)
        for i in range(len(maccs_fp)):
            feature_rows[f"MACCS_{i}"] = int(maccs_fp[i])

        # AtomPair Fingerprint (2D)
        atompair_fp = atom_gen.GetCountFingerprint(mol)
        for i in range(morgan_nbits):
            feature_rows[f"AtomPair_{i}"] = atompair_fp[i]

        # RDKIT FINGERPRINT
        rdkit_fp = RDKFingerprint(mol)
        for i in range(len(rdkit_fp)):
            feature_rows[f"RDKIT_{i}"] = int(rdkit_fp[i])

        # AVALON FINGERPRINT (IF AVAILABLE) 
        if avalon_available:
            avalon_fp = pyAvalonTools.GetAvalonFP(mol, morgan_nbits)
        for i in range(len(avalon_fp)):
            feature_rows[f"Avalon_{i}"] = int(avalon_fp[i])


        fps_data.append(feature_rows)

    print(f'There are {morgan_nbits} Morgan Fingerprint Features')
    print(f'There are {len(maccs_fp)} MACCS Keys Features')
    print(f'There are {len(rdkit_fp)} RDKIT Fingerprint Features')

    # MERGE REAL DATA WITH EXTRACTED FEATURES
    fps_df = pd.DataFrame(fps_data)
    df_result = pd.concat((df, fps_df), axis = 1)

    return df_result


# APPLY FUNCTION
merge_df = extract_all_fingerprint(merge_df, 'SMILES')
test_df  = extract_all_fingerprint(test_df, 'SMILES')

merge_df

# ==========================================================================================
# ==================================== DATA SPLITTING ======================================
# ==========================================================================================

# SPLIT DATA
x = merge_df.drop(labels = ['SMILES', 'Tm'], axis = 1)
y = merge_df['Tm']

x_test = test_df.drop(labels = ['SMILES', 'id'], axis = 1)

x.shape, y.shape, x_test.shape, type(x)

# ==========================================================================================
# =========================== OPTUNA HYPERPARAMETER OPTIMIZATION ===========================
# ==========================================================================================

def objective(trial):
    
    xgb_params = {
        "verbosity": 0,
        "objective": trial.suggest_categorical("objective", ['reg:squarederror', 'reg:pseudohubererror']),
        "tree_method": "gpu_hist",
        'predictor' : 'gpu_predictor',
        'device' : 'cuda',
        "eval_metric": "rmse",
        "booster": "gbtree",
        'n_estimators' : 10_000,
        "max_depth": trial.suggest_int("max_depth", 3, 7),
        "learning_rate": trial.suggest_float("learning_rate", 3e-3, 0.3, log=True),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "lambda": trial.suggest_float("lambda", 0.1, 20.0, log=True),
        "alpha": trial.suggest_float("alpha", 0.1, 20.0, log=True),
    }

    kf = KFold(n_splits=5, shuffle=True, random_state = 2025)
    rmse_scores = []

    for train_idx, valid_idx in kf.split(x):
        X_train, X_valid = x.iloc[train_idx], x.iloc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        dtrain = xgboost.DMatrix(X_train, label=y_train)
        dvalid = xgboost.DMatrix(X_valid, label=y_valid)

        model = xgboost.train(
            xgb_params,
            dtrain,
            num_boost_round=10000,
            evals=[(dvalid, "validation")],
            early_stopping_rounds=100,
            verbose_eval=False,
        )

        preds = model.predict(dvalid)
        rmse = mean_squared_error(y_valid, preds, squared = False)
        rmse_scores.append(rmse)

    return np.mean(rmse_scores)

# START OPTUNA

# MOVE AND COPY FILE TO OUPUT
shutil.copy(src = '/kaggle/input/optuna-study-3-models/other/optuna-study-3-models/2/xgb_study.db', 
            dst = '/kaggle/working/xgb_study.db')

study = optuna.create_study(direction="minimize", study_name = 'xgb_study', 
                            storage="sqlite://////kaggle/working/xgb_study.db", 
                            load_if_exists = True)

# CLEAR OUTPUT AFTER TRAINING 
clear_output(wait=True)

print(f'Training Complete! Congrats!')
print(f'Total Number of Trials : {len(study.trials)}\n')

print("Best Trial", study.best_trial.number)
print("Best MAE:", study.best_value)
print("Best Params:", study.best_trial.params)

# ==========================================================================================
# ============================ MODEL CONFIGURATION AND PARAMS ==============================
# ==========================================================================================

# TRAIN XGB WITH BEST PARAMETERS

#best_params = study.best_trial.params
#best_params.update({'eval_metric' : "mae",
#                    'device' : 'cpu'})

# USE SIMPLER MODEL THAN TUNED MODEL (DUE TO SLOW COMPUTATION)
best_params = {
    'max_depth' : 6,
    'eta' : 0.1,
    'tree_method' : 'hist',
    'eval_metric' : 'mae'
}

best_params

'''{'objective': 'reg:squarederror',
 'max_depth': 3,
 'learning_rate': 0.01688021212211354,
 'min_child_weight': 5,
 'subsample': 0.8943127227676447,
 'colsample_bytree': 0.590582609011384,
 'gamma': 0.01132850232872052,
 'lambda': 4.445806747037075,
 'alpha': 0.1292033669407927,
 'eval_metric': 'mae',
 'device': 'cpu'}'''

# ==========================================================================================
# ============================== CROSS-VALIDATION TRAINING =================================
# ==========================================================================================

# XGB WITH SKFOLD

skfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 2025)

# DEFINE YEO-JOHNSON FOR TRANSFORMING TARGET FEATURE INSIDE LOOP
yeo = PowerTransformer(method = 'yeo-johnson')

# STORE OOF AND TEST MAE
oof_val = np.zeros(len(x))
train_score , val_score, test_pred = [], [], []

for i, (train_index, val_index) in enumerate(skfold.split(x, pd.qcut(y, q = 10).cat.codes)):
    
    # SPLIT DATA
    x_train, x_val = x.iloc[train_index], x.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # USING YEO-JOHNSON TO TRANSFORM TARGET FEATURE
    y_train = yeo.fit_transform(y_train.values.reshape(-1, 1)).squeeze()
    y_val   = yeo.transform(y_val.values.reshape(-1, 1)).squeeze()

    # BUILD TINY CATBOOST TO GET THE MOST IMPORTANT FEATURE ONLY
    selector_model = xgboost.XGBRegressor(
        n_estimators=500,
        max_depth = 6,
        learning_rate=0.05,
        random_state=2025,
        device = 'cpu',
        objective="reg:absoluteerror",
        tree_method = 'hist',
        verbosity=0
    )

    # GET THE MOST IMPORTANT FEATURE ONLY 
    selector_model.fit(x_train, y_train)
    selector = SelectFromModel(selector_model, prefit=True, threshold="mean")

    # CHECK AND DISPLAY HOW MANY IMPORTANT FEATURES ARE
    selected_idx = selector.get_support(indices=True) 
    selected_features = x_train.columns[selected_idx]
    print("Selected features:", len(selected_features),"\n")

    # APPLY TRANSFORM (ONLY GET MOST IMPORTANT FEATURE AND REMOVE USELESS FEATURES)
    x_train_new = x_train[selected_features]
    x_val_new   = x_val[selected_features]
    x_test_new  = x_test[selected_features]

    # XGBOOST DATASET
    dtrain = xgboost.DMatrix(x_train_new, y_train, feature_names = selected_features.tolist())
    dval   = xgboost.DMatrix(x_val_new, y_val, feature_names = selected_features.tolist())
    dtest  = xgboost.DMatrix(x_test_new, feature_names = selected_features.tolist())

    # XGBOOST
    xgb = xgboost.train(params = best_params, 
                        dtrain = dtrain, 
                        num_boost_round = 15_000, 
                        evals = [(dtrain, 'train'), (dval, 'valid')],
                        early_stopping_rounds = 300,
                        verbose_eval = 1000)
    
    # DISPLAY SELECTED IMPORTANCE FEATURES
    fig, ax = plt.subplots(figsize=(10, 7))
    xgboost.plot_importance(booster = xgb, importance_type = 'gain', max_num_features = 30, ax = ax)
    plt.title(f'Feature Importance fold {i+1}')
    plt.show()

    # PREDICT
    y_train_predict = xgb.predict(dtrain)
    y_val_predict   = xgb.predict(dval)
    y_test_predict = xgb.predict(dtest)

    # INVERSE TRANSFORM (TRANSFORM A VALUE BACK TO NORMAL)
    y_train = yeo.inverse_transform(y_train.reshape(-1, 1)).squeeze()
    y_val   = yeo.inverse_transform(y_val.reshape(-1, 1)).squeeze()
    y_train_predict = yeo.inverse_transform(y_train_predict.reshape(-1, 1)).squeeze()
    y_val_predict   = yeo.inverse_transform(y_val_predict.reshape(-1, 1)).squeeze()
    y_test_predict  = yeo.inverse_transform(y_test_predict.reshape(-1, 1)).squeeze()

    # MAE
    train_mae = mean_absolute_error(y_train, y_train_predict)
    val_mae   = mean_absolute_error(y_val, y_val_predict)

    print(f'Fold {i+1} : Train MAE = {train_mae}, Val MAE = {val_mae}')

    train_score.append(train_mae)
    val_score.append(val_mae)

    # PUSH OOF PREDICTION AND TEST PREDICTION
    oof_val[val_index] = y_val_predict
    test_pred.append(y_test_predict)


print(f'\nTrain Fold Prediction : {np.mean(train_score)}')
print(f'Val Fold Prediction   : {np.mean(val_score)}\n')

print(f'std Train Fold Prediction : {np.std(train_score, ddof = 0)}')
print(f'std Val Fold Prediction   : {np.std(val_score, ddof = 0)}')

# ==========================================================================================
# ============================== EVALUATION AND VISUALIZATION ==============================
# ==========================================================================================

# ACTUAL VS PREDICTED LABEL

y_true = merge_df['Tm'].values
y_pred = oof_val

plt.figure(figsize=(6, 4))

# SCATTER PLOT 2D
plt.scatter(y_pred, y_true, alpha=0.7)
plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'k--', lw=2)
plt.xlabel('Predicted values')
plt.ylabel('Actual values')
plt.title('Actual vs Predicted')

plt.show()

# ==========================================================================================
# ============================== OPTUNA PLOTS AND ANALYSIS =================================
# ==========================================================================================

# SHOW OPTIMIZATION HISTORY 
optuna.visualization.plot_optimization_history(study) # --> SHOW THE BEST VALUE LINE DURING STUDY

# ANALYZE HYPERPARAMETER INTERACTIONS
optuna.visualization.plot_parallel_coordinate(study)

# DISPLAY SENSITIVY EACH PARAMETER
optuna.visualization.plot_slice(study)

# DISPLAY DECISION BOUNDARY 
fig = optuna.visualization.plot_contour(study, params=['alpha',
                                                 'colsample_bytree',
                                                 'gamma',
                                                 'lambda',
                                                 'learning_rate',
                                                 'max_depth'])
fig.update_layout(width = 2000, height = 900)

# EMPIRICAL DISTRIBUTION
optuna.visualization.plot_edf(study) # DISPLAY PROBABILITY OF OBJECTIVE VALUE IS LESS THAN OR EQUAL TO A GIVEN THRESHOLD (EDF CURVE)

# DISPLAY HYPERPARAMETER IMPORTANCES
optuna.visualization.plot_param_importances(study)

# ==========================================================================================
# ====================================== SUBMISSION ========================================
# ==========================================================================================

# SUBMISSION

submission = pd.read_csv(r'/kaggle/input/melting-point/sample_submission.csv')

submission['Tm'] = np.mean(test_pred, axis = 0)

submission

# SAVE SUBMISSION
submission.to_csv(r'submission_xgb_176460.csv', index = False)

# SAVE OOF PREDICTION

oof_df = pd.DataFrame(oof_val) # --> CONVERT TO DATAFRAME

display(oof_df)

# SAVE OOF
oof_df.to_csv(r'oof_xgb_176460.csv', index = False)